# Python Spark SQL 

Ada beberapa step yang dilakukan yaitu:



1.   **Install Spark pada Google Colab**
2.   **Create PySpark DataFrame** 
3.   **Inner Join PySpark**
4.   **Query PySpark**
5.   **Aggregation**
6. **SQL Query**

# Installing Spark on Google Colab

In [1]:
# Checking folder
!ls

sample_data


In [2]:
# Installing JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# Getting Spark installer (check the path on spark.apache.org)
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

In [4]:
# Checking if the file is copied
!ls

sample_data  spark-3.2.1-bin-hadoop2.7.tgz


In [5]:
# Untar the Spark installer
!tar -xvf spark-3.2.1-bin-hadoop2.7.tgz

spark-3.2.1-bin-hadoop2.7/
spark-3.2.1-bin-hadoop2.7/LICENSE
spark-3.2.1-bin-hadoop2.7/NOTICE
spark-3.2.1-bin-hadoop2.7/R/
spark-3.2.1-bin-hadoop2.7/R/lib/
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/DESCRIPTION
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/Meta/Rd.rds
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/Meta/features.rds
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/Meta/hsearch.rds
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/Meta/links.rds
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/Meta/nsInfo.rds
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/Meta/package.rds
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/NAMESPACE
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/R/
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/R/SparkR
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdb
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdx
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/help/
spark-3.2.1-bin-hadoop2.7/R/lib/SparkR/help/An

In [6]:
# Checking the Spark folder after untar
!ls 

sample_data  spark-3.2.1-bin-hadoop2.7	spark-3.2.1-bin-hadoop2.7.tgz


In [7]:
# Installing findspark - a python library to find Spark
!pip install -q findspark

In [8]:
# Setting environment variables: Setting Java and Spark home based on the location where they are stored
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [9]:
# Creating a local Spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Loading Data

In [10]:
# Loading data observations in pandas dataframe
import pandas as pd
observation_df = pd.read_csv("https://raw.githubusercontent.com/laksmitawidya/fhir-project/master/observations.csv") 
observation_df.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8302-2,Body Height,193.3,cm,numeric
1,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
2,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,29463-7,Body Weight,87.8,kg,numeric
3,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,39156-5,Body Mass Index,23.5,kg/m2,numeric
4,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8462-4,Diastolic Blood Pressure,82.0,mm[Hg],numeric


In [11]:
# Loading data encounter in pandas dataframe
import pandas as pd
encounter_df = pd.read_csv("https://raw.githubusercontent.com/laksmitawidya/fhir-project/master/encounters.csv") 
encounter_df.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,185345009,Encounter for symptom,129.16,129.16,54.16,10509002.0,Acute bronchitis (disorder)
1,e88bc3a9-007c-405e-aabc-792a38f4aa2b,2012-01-23T17:45:28Z,2012-01-23T18:00:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,772ee193-bb9f-30eb-9939-21e86c8e4da5,6f1d59a7-a5bd-3cf9-9671-5bad2f351c28,6e2f1a2d-27bd-3701-8d08-dae202c58632,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
2,8f104aa7-4ca9-4473-885a-bba2437df588,2001-05-01T15:02:18Z,2001-05-01T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,5d4b9df1-93ae-3bc9-b680-03249990e558,af01a385-31d3-3c77-8fdb-2867fe88df2f,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185345009,Encounter for symptom,129.16,129.16,0.00,36971009.0,Sinusitis (disorder)
3,b85c339a-6076-43ed-b9d0-9cf013dec49d,2011-07-28T15:02:18Z,2011-07-28T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,3dc9bb2d-5d66-3e61-bf9a-e234c6433577,bb17e691-262b-3546-93d5-d88e7de93246,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,162673000,General examination of patient (procedure),129.16,129.16,0.00,NaN,NaN
4,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,2010-07-27T12:58:08Z,2010-07-27T13:28:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,b03dba4f-892f-365c-bfd1-bfcfa7a98d5d,7ed6b84a-b847-3744-9d42-15c42297a0c2,d47b3510-2895-3b70-9897-342d681c769d,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN


In [12]:
# Loading data patient in pandas dataframe
import pandas as pd
patients_df = pd.read_csv("https://raw.githubusercontent.com/laksmitawidya/fhir-project/master/patients.csv") 
patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,NaN,999-76-6866,S99984236,X19277260X,Mr.,José Eduardo181,Gómez206,NaN,...,Marigot Saint Andrew Parish DM,427 Balistreri Way Unit 19,Chicopee,Massachusetts,Hampden County,1013.0,42.228354,-72.562951,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,NaN,999-73-5361,S99962402,X88275464X,Mr.,Milo271,Feil794,NaN,...,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,NaN,999-27-3385,S99972682,X73754411X,Mr.,Jayson808,Fadel536,NaN,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,NaN,999-85-4926,S99974448,X40915583X,Mrs.,Mariana775,Rutherford999,NaN,...,Yarmouth Massachusetts US,999 Kuhn Forge,Lowell,Massachusetts,Middlesex County,1851.0,42.636143,-71.343255,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,NaN,999-60-7372,S99915787,X86772962X,Mr.,Gregorio366,Auer97,NaN,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,598763.07,3772.20


In [13]:
# Dropping all null rows & change NaN to 0
observation_df.dropna(how="all", inplace=True)
observation_df.fillna(0)
observation_df.shape

encounter_df.dropna(how="all", inplace=True)
encounter_df.fillna(0)
encounter_df.shape

patients_df.dropna(how="all", inplace=True)
patients_df.fillna(0)
patients_df.shape

(1171, 25)

In [14]:
# Replacing carriage return and new line characters with a space
observation_df = observation_df.replace({r'\r\n': ' '}, regex=True)
observation_df.head()

encounter_df = encounter_df.replace({r'\r\n': ' '}, regex=True)
encounter_df.head()

patients_df = patients_df.replace({r'\r\n': ' '}, regex=True)
patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,NaN,999-76-6866,S99984236,X19277260X,Mr.,José Eduardo181,Gómez206,NaN,...,Marigot Saint Andrew Parish DM,427 Balistreri Way Unit 19,Chicopee,Massachusetts,Hampden County,1013.0,42.228354,-72.562951,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,NaN,999-73-5361,S99962402,X88275464X,Mr.,Milo271,Feil794,NaN,...,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,NaN,999-27-3385,S99972682,X73754411X,Mr.,Jayson808,Fadel536,NaN,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,NaN,999-85-4926,S99974448,X40915583X,Mrs.,Mariana775,Rutherford999,NaN,...,Yarmouth Massachusetts US,999 Kuhn Forge,Lowell,Massachusetts,Middlesex County,1851.0,42.636143,-71.343255,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,NaN,999-60-7372,S99915787,X86772962X,Mr.,Gregorio366,Auer97,NaN,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,598763.07,3772.20


# Create PySpark DataFrame

In [15]:
# Converting Pandas Dataframe into Spark Dataframe
encounter_df = encounter_df.astype(str) # Converting pandas df to string first
encounter_df.dtypes
encounter_sdf = spark.createDataFrame(encounter_df)
encounter_sdf.show(10, False) # False allows us to show entire content of the columns

+------------------------------------+--------------------+--------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+--------------+---------+------------------------------------------+-------------------+----------------+--------------+-----------+----------------------------------+
|Id                                  |START               |STOP                |PATIENT                             |ORGANIZATION                        |PROVIDER                            |PAYER                               |ENCOUNTERCLASS|CODE     |DESCRIPTION                               |BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|PAYER_COVERAGE|REASONCODE |REASONDESCRIPTION                 |
+------------------------------------+--------------------+--------------------+------------------------------------+------------------------------------+------------------------------------+-----------------

In [16]:
# Converting Pandas Dataframe into Spark Dataframe
observation_df = observation_df.astype(str) # Converting pandas df to string first
observation_df.dtypes
observation_sdf = spark.createDataFrame(observation_df)
observation_sdf.show(10, False) # False allows us to show entire content of the columns

+--------------------+------------------------------------+------------------------------------+-------+-------------------------------------------------------------+-----+-------+-------+
|DATE                |PATIENT                             |ENCOUNTER                           |CODE   |DESCRIPTION                                                  |VALUE|UNITS  |TYPE   |
+--------------------+------------------------------------+------------------------------------+-------+-------------------------------------------------------------+-----+-------+-------+
|2012-01-23T17:45:28Z|034e9e3b-2def-4559-bb2a-7850888ae060|e88bc3a9-007c-405e-aabc-792a38f4aa2b|8302-2 |Body Height                                                  |193.3|cm     |numeric|
|2012-01-23T17:45:28Z|034e9e3b-2def-4559-bb2a-7850888ae060|e88bc3a9-007c-405e-aabc-792a38f4aa2b|72514-3|Pain severity - 0-10 verbal numeric rating [Score] - Reported|2.0  |{score}|numeric|
|2012-01-23T17:45:28Z|034e9e3b-2def-4559-bb2a-7850888ae

In [17]:
# Converting Pandas Dataframe into Spark Dataframe
patients_df = patients_df.astype(str) # Converting pandas df to string first
patients_df.dtypes
patients_sdf = spark.createDataFrame(patients_df)
patients_sdf.show(10, False) # False allows us to show entire content of the columns

+------------------------------------+----------+---------+-----------+---------+----------+------+---------------+-------------+------+-------------+-------+-----+-----------+------+--------------------------------+------------------------------+-----------+-------------+----------------+------+------------------+------------------+-------------------+-------------------+
|Id                                  |BIRTHDATE |DEATHDATE|SSN        |DRIVERS  |PASSPORT  |PREFIX|FIRST          |LAST         |SUFFIX|MAIDEN       |MARITAL|RACE |ETHNICITY  |GENDER|BIRTHPLACE                      |ADDRESS                       |CITY       |STATE        |COUNTY          |ZIP   |LAT               |LON               |HEALTHCARE_EXPENSES|HEALTHCARE_COVERAGE|
+------------------------------------+----------+---------+-----------+---------+----------+------+---------------+-------------+------+-------------+-------+-----+-----------+------+--------------------------------+------------------------------+-

# Inner Join PySpark

In [18]:
# join 3 dataframes
df = observation_sdf.join(encounter_sdf,(observation_sdf.ENCOUNTER == encounter_sdf.Id) & (observation_sdf.PATIENT == encounter_sdf.PATIENT),"inner").join(patients_sdf,(observation_sdf.PATIENT == patients_sdf.Id)&(encounter_sdf.PATIENT == patients_sdf.Id) & (observation_sdf.PATIENT == encounter_sdf.PATIENT),"inner")

In [19]:
df.show()

+--------------------+--------------------+--------------------+-------+--------------------+------------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+-------------------+----------------+--------------+----------+-----------------+--------------------+----------+----------+-----------+---------+----------+------+---------+---------+------+-----------+-------+------+-----------+------+--------------------+--------------------+-------+-------------+-----------------+---+-----------------+------------------+-------------------+-------------------+
|                DATE|             PATIENT|           ENCOUNTER|   CODE|         DESCRIPTION|       VALUE|  UNITS|   TYPE|                  Id|               START|                STOP|             PATIENT|        ORGANIZATION|            PROVIDER|               PAYER|ENCOUNTERCLASS|

In [20]:
df.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- ENCOUNTER: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- VALUE: string (nullable = true)
 |-- UNITS: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- START: string (nullable = true)
 |-- STOP: string (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- ORGANIZATION: string (nullable = true)
 |-- PROVIDER: string (nullable = true)
 |-- PAYER: string (nullable = true)
 |-- ENCOUNTERCLASS: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- BASE_ENCOUNTER_COST: string (nullable = true)
 |-- TOTAL_CLAIM_COST: string (nullable = true)
 |-- PAYER_COVERAGE: string (nullable = true)
 |-- REASONCODE: string (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- BIRTHDATE: string (null

# Query PySpark

In [21]:
df.select(encounter_sdf.PATIENT.alias("patient"), "GENDER", observation_sdf.CODE.alias("code"), "COUNTY", "HEALTHCARE_COVERAGE", "ENCOUNTER", encounter_sdf.DESCRIPTION.alias("Encounter Desc")).show()

+--------------------+------+-------+----------------+-------------------+--------------------+--------------------+
|             patient|GENDER|   code|          COUNTY|HEALTHCARE_COVERAGE|           ENCOUNTER|      Encounter Desc|
+--------------------+------+-------+----------------+-------------------+--------------------+--------------------+
|01207ecd-9dff-475...|     F| 8302-2| Franklin County|  774.9599999999999|b1db8760-cdd7-4bb...|Well child visit ...|
|01207ecd-9dff-475...|     F|72514-3| Franklin County|  774.9599999999999|b1db8760-cdd7-4bb...|Well child visit ...|
|01207ecd-9dff-475...|     F|29463-7| Franklin County|  774.9599999999999|b1db8760-cdd7-4bb...|Well child visit ...|
|01207ecd-9dff-475...|     F|77606-2| Franklin County|  774.9599999999999|b1db8760-cdd7-4bb...|Well child visit ...|
|01207ecd-9dff-475...|     F| 9843-4| Franklin County|  774.9599999999999|b1db8760-cdd7-4bb...|Well child visit ...|
|01207ecd-9dff-475...|     F| 8462-4| Franklin County|  774.9599

In [22]:
df.withColumn("HEALTHCARE_EXPENSES",df.HEALTHCARE_EXPENSES.cast('double'))

DataFrame[DATE: string, PATIENT: string, ENCOUNTER: string, CODE: string, DESCRIPTION: string, VALUE: string, UNITS: string, TYPE: string, Id: string, START: string, STOP: string, PATIENT: string, ORGANIZATION: string, PROVIDER: string, PAYER: string, ENCOUNTERCLASS: string, CODE: string, DESCRIPTION: string, BASE_ENCOUNTER_COST: string, TOTAL_CLAIM_COST: string, PAYER_COVERAGE: string, REASONCODE: string, REASONDESCRIPTION: string, Id: string, BIRTHDATE: string, DEATHDATE: string, SSN: string, DRIVERS: string, PASSPORT: string, PREFIX: string, FIRST: string, LAST: string, SUFFIX: string, MAIDEN: string, MARITAL: string, RACE: string, ETHNICITY: string, GENDER: string, BIRTHPLACE: string, ADDRESS: string, CITY: string, STATE: string, COUNTY: string, ZIP: string, LAT: string, LON: string, HEALTHCARE_EXPENSES: double, HEALTHCARE_COVERAGE: string]

In [23]:
df.filter(df['HEALTHCARE_EXPENSES'] > 2000000).show()

+--------------------+--------------------+--------------------+-------+--------------------+-----+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+-------------------+----------------+--------------+----------+-----------------+--------------------+----------+----------+-----------+---------+----------+------+---------+--------+------+------+-------+-----+-----------+------+--------------------+--------------------+---------+-------------+----------------+---+------------------+------------------+-------------------+-------------------+
|                DATE|             PATIENT|           ENCOUNTER|   CODE|         DESCRIPTION|VALUE|  UNITS|   TYPE|                  Id|               START|                STOP|             PATIENT|        ORGANIZATION|            PROVIDER|               PAYER|ENCOUNTERCLASS|     CODE|         

In [40]:
df.sort("HEALTHCARE_EXPENSES", ascending=True).show()

+--------------------+--------------------+--------------------+-------+--------------------+------------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+-------------------+----------------+--------------+----------+-----------------+--------------------+----------+---------+-----------+---------+----------+------+--------+---------+------+------+-------+-----+-----------+------+--------------------+--------------------+-------+-------------+----------------+---+------------------+------------------+-------------------+-------------------+
|                DATE|             PATIENT|           ENCOUNTER|   CODE|         DESCRIPTION|       VALUE|  UNITS|   TYPE|                  Id|               START|                STOP|             PATIENT|        ORGANIZATION|            PROVIDER|               PAYER|ENCOUNTERCLASS|     COD

In [54]:
df.filter(df.DEATHDATE.isNotNull()).select("DEATHDATE").distinct().show()

+----------+
| DEATHDATE|
+----------+
|1990-03-17|
|1996-10-02|
|2012-01-12|
|2013-08-20|
|2006-01-08|
|2005-08-15|
|1999-03-25|
|2017-05-18|
|1974-10-01|
|1977-03-30|
|1975-11-03|
|2003-02-15|
|1991-04-30|
|2016-01-30|
|1990-03-07|
|1995-06-14|
|2017-12-29|
|2014-07-10|
|2009-06-20|
|2003-01-06|
+----------+
only showing top 20 rows



In [57]:
df.orderBy(df.HEALTHCARE_EXPENSES.desc()).select("HEALTHCARE_EXPENSES").show()

+-------------------+
|HEALTHCARE_EXPENSES|
+-------------------+
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
|          999894.46|
+-------------------+
only showing top 20 rows



In [53]:
df[df.RACE.isin(["white", "native"])].select("RACE").distinct().show()

+------+
|  RACE|
+------+
| white|
|native|
+------+



In [36]:
df.distinct().count()

268892

# Aggregation

In [24]:
df.groupBy("ENCOUNTERCLASS").count().show()

+--------------+------+
|ENCOUNTERCLASS| count|
+--------------+------+
|      wellness|151791|
|    outpatient| 40035|
|     emergency|  1528|
|     inpatient| 16377|
|    ambulatory| 59603|
+--------------+------+



In [25]:
from pyspark.sql.functions import max, avg, min
# Find the costliest book
df.agg(max("HEALTHCARE_EXPENSES").alias("Max Health Expenses"), avg("HEALTHCARE_EXPENSES").alias("Avg Health Expenses"), min("HEALTHCARE_EXPENSES").alias("Min Health Expenses")).show()

+-------------------+-------------------+-------------------+
|Max Health Expenses|Avg Health Expenses|Min Health Expenses|
+-------------------+-------------------+-------------------+
|          999894.46| 1034357.2269438703|          1003575.1|
+-------------------+-------------------+-------------------+



# SQL Query

In [26]:
df.createOrReplaceTempView("fhir")

sqlDF = spark.sql("SELECT * FROM fhir")
sqlDF.show()

+--------------------+--------------------+--------------------+-------+--------------------+------------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+-------------------+----------------+--------------+----------+-----------------+--------------------+----------+----------+-----------+---------+----------+------+---------+---------+------+-----------+-------+------+-----------+------+--------------------+--------------------+-------+-------------+-----------------+---+-----------------+------------------+-------------------+-------------------+
|                DATE|             PATIENT|           ENCOUNTER|   CODE|         DESCRIPTION|       VALUE|  UNITS|   TYPE|                  Id|               START|                STOP|             PATIENT|        ORGANIZATION|            PROVIDER|               PAYER|ENCOUNTERCLASS|